In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, confusion_matrix, ConfusionMatrixDisplay,
    RocCurveDisplay, classification_report)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
test_df = pd.read_csv('/content/drive/My Drive/Data/Capstone/test.csv')
train_df = pd.read_csv('/content/drive/My Drive/Data/Capstone/train.csv')
BTS_df = pd.read_csv('/content/drive/My Drive/Data/Capstone/BTS_Data.csv')

In [26]:
test_df.info()
train_df.info()
BTS_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275 entries, 0 to 274
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Date                  275 non-null    object
 1   Region                275 non-null    object
 2   Brand                 275 non-null    object
 3   Model                 275 non-null    object
 4   Vehicle_Type          275 non-null    object
 5   Battery_Capacity_kWh  275 non-null    int64 
 6   Discount_Percentage   275 non-null    int64 
 7   Customer_Segment      275 non-null    object
 8   Fast_Charging_Option  275 non-null    object
dtypes: int64(2), object(7)
memory usage: 19.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531 entries, 0 to 530
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Date                  531 non-null    object
 1   Region                531 non-null 

In [27]:
print(test_df.isnull().sum())
print(train_df.isnull().sum())
print(BTS_df.isnull().sum())

Date                    0
Region                  0
Brand                   0
Model                   0
Vehicle_Type            0
Battery_Capacity_kWh    0
Discount_Percentage     0
Customer_Segment        0
Fast_Charging_Option    0
dtype: int64
Date                    0
Region                  0
Brand                   0
Model                   0
Vehicle_Type            0
Battery_Capacity_kWh    0
Discount_Percentage     0
Customer_Segment        0
Fast_Charging_Option    0
Units_Sold              0
Revenue                 0
dtype: int64
Table 1-19:  Hybrid-Electric, Plug-in Hybrid-Electric, and Electric Vehicle Sales    3
Unnamed: 1                                                                           9
Unnamed: 2                                                                           9
Unnamed: 3                                                                           9
Unnamed: 4                                                                           9
Unnamed: 5         

In [31]:
train_df.dropna()
test_df.dropna()
BTS_df.dropna()

,"Table 1-19: Hybrid-Electric, Plug-in Hybrid-Electric, and Electric Vehicle Sales",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
1,Hybrid electric,"9,350","20,282","36,042","47,566","84,233","205,876","251,864","351,071","315,763",...,"384,400","346,949","370,680","338,149","399,444","455,067","799,046","766,412","1,175,456","1,609,035"
2,Plug-in hybrid-electric,Z,Z,Z,Z,Z,Z,Z,Z,Z,...,"43,008","72,885","91,188","123,883","85,791","69,049","176,117","183,411","293,578","321,774"
3,Electric,Z,Z,Z,Z,Z,Z,Z,Z,Z,...,"71,064","86,731","104,487","207,062","233,822","238,540","459,474","747,982","1,164,638","1,247,656"


In [33]:
# change to numeric
train_df = train_df.apply(pd.to_numeric, errors='ignore')
test_df = test_df.apply(pd.to_numeric, errors='ignore')
BTS_df = BTS_df.apply(pd.to_numeric, errors='ignore')

/tmp/ipython-input-910502900.py:1: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  train_df = train_df.apply(pd.to_numeric, errors='ignore')
/tmp/ipython-input-910502900.py:2: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  test_df = test_df.apply(pd.to_numeric, errors='ignore')
/tmp/ipython-input-910502900.py:3: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  BTS_df = BTS_df.apply(pd.to_numeric, errors='ignore')


In [35]:
# averages
num_cols = train_df.select_dtypes(include='number').columns

train_df['numeric_total'] = train_df[num_cols].sum(axis=1)
train_df['numeric_avg'] = train_df[num_cols].mean(axis=1)

In [36]:
# indicator value added
train_df['high_total_flag'] = (train_df['numeric_total'] > train_df['numeric_total'].median()).astype(int)